In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pandas as pd
from gensim.models import Word2Vec
import ast
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.layers import Input, Dense, Concatenate
from keras.models import Model

movies = pd.read_csv('../dataset/movies_metadata.csv')
ratings = pd.read_csv('../dataset/ratings.csv')

C:\Users\ssy30\AppData\Local\Temp\ipykernel_40068\2339703475.py:19: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('../dataset/movies_metadata.csv')


In [2]:
ratings

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556
...,...,...,...,...
26024284,270896,58559,5.0,1257031564
26024285,270896,60069,5.0,1257032032
26024286,270896,63082,4.5,1257031764
26024287,270896,64957,4.5,1257033990


In [3]:
movies

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [4]:
# Filter out rows with missing genres
movies = movies[movies['genres'].notnull()]

# Convert genres from JSON to list
movies['genres'] = movies['genres'].apply(lambda x: [genre['name'] for genre in eval(x)])

# Join genres into a single string
movies['genres'] = movies['genres'].apply(lambda x: ' '.join(x))

In [5]:
# Create a TF-IDF Vectorizer to tokenize genres and calculate TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(movies['genres'])

# Calculate cosine similarity between movies based on genres
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [6]:

# Function to recommend movies based on user's watched movies
def recommend_movies(watched_movies, movies, cosine_sim):
    recommended_movies = []
    for movie_idx in watched_movies:
        # Get similarity scores for the watched movie
        sim_scores = list(enumerate(cosine_sim[movie_idx]))
        # Sort movies based on similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        # Get top 3 similar movies (excluding the watched movie itself)
        sim_scores = sim_scores[1:4]
        # Get movie indices
        movie_indices = [x[0] for x in sim_scores]
        # Get movie titles
        recommended_movies.extend([movies.iloc[i]['title'] for i in movie_indices])
    return recommended_movies


In [7]:
# Example user watched movies (using index from the dataset)
watched_movies = [0, 1]  # Indices of movies "Toy Story" and "Jumanji"

# Recommend movies based on user's watched movies
recommended_movies = recommend_movies(watched_movies, movies, cosine_sim)

In [8]:
# Print recommended movies
print("Recommended Movies:")
for movie in recommended_movies:
    print(movie)


# End of just cosine similarity

Recommended Movies:
Oliver & Company
A Close Shave
The Wrong Trousers
The Indian in the Cupboard
The Wizard of Oz
Labyrinth


In [2]:

# Clean 'id' column in movies DataFrame
movies['id'] = movies['id'].str.replace(r'\D', '').astype('int64')

# Merge ratings with movie metadata
ratings = pd.merge(ratings, movies[['id', 'title']], left_on='movieId', right_on='id')

# Aggregate ratings by movie and calculate mean rating
movie_ratings = ratings.groupby('title')['rating'].mean().reset_index()

# Merge mean ratings with original ratings data
ratings = pd.merge(ratings, movie_ratings, on='title', suffixes=('', '_mean'))


C:\Users\ssy30\AppData\Local\Temp\ipykernel_40068\439772099.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  movies['id'] = movies['id'].str.replace(r'\D', '').astype('int64')


In [3]:
# Load keywords.csv into a DataFrame
keywords_df = pd.read_csv('../dataset/keywords.csv')

# Convert the string representation of the list of dictionaries to an actual list of dictionaries
keywords_df['keywords'] = keywords_df['keywords'].apply(lambda x: ast.literal_eval(x))

# Train a Word2Vec model on the keywords
sentences = keywords_df['keywords'].apply(lambda x: [word_dict['name'] for word_dict in x]).tolist()

# Use the merged DataFrame in your recommendation model
# (this step depends on your recommendation model architecture)



In [4]:
credits_df = pd.read_csv('../dataset/credits.csv')

In [5]:
# Function to extract cast names from a row and concatenate them into a single string
def extract_cast_names(row):
    cast_list = ast.literal_eval(row['cast'])
    cast_names = [cast['name'] for cast in cast_list]
    return ', '.join(cast_names)

# Apply the function to each row to create a new column 'cast_names' containing the concatenated names
credits_df['cast_names'] = credits_df.apply(extract_cast_names, axis=1)

credits_df

,cast,crew,id,cast_names
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,..."
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ..."
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph..."
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,"Whitney Houston, Angela Bassett, Loretta Devin..."
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,"Steve Martin, Diane Keaton, Martin Short, Kimb..."
...,...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050,"Leila Hatami, Kourosh Tahami, Elham Korda"
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109,"Angel Aquino, Perry Dizon, Hazel Orencio, Joel..."
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758,"Erika Eleniak, Adam Baldwin, Julie du Page, Ja..."
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506,"Iwan Mosschuchin, Nathalie Lissenko, Pavel Pav..."


In [6]:
# Train a Word2Vec model on the keywords
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, sg=1)

# Function to get the average embedding for a list of keywords
def get_avg_embedding(keywords):
    embeddings = [model.wv[word] for word in keywords if word in model.wv]
    if embeddings:
        return sum(embeddings) / len(embeddings)
    else:
        return [0] * model.vector_size

# Apply the function to the 'keywords' column to get the average embedding for each movie
keywords_df['keyword_embeddings'] = keywords_df['keywords'].apply(lambda x: get_avg_embedding([word_dict['name'] for word_dict in x]))

# Merge the keyword embeddings with the original DataFrame (assuming 'id' is the movie ID column)
keywords_df = keywords_df[['id', 'keyword_embeddings']]
merged_df = pd.merge(ratings, keywords_df, left_on='movieId', right_on='id')


In [7]:
sentences = [row.split() for row in credits_df['cast_names']]
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, sg=1)

# Function to get the average embedding for a list of keywords
def get_avg_embedding_cast(keywords):
    embeddings = [model.wv[keyword] for keyword in keywords if keyword in model.wv]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(100)  # Return zero vector if no embeddings found

# Apply the function to the keywords data and create a new column 'keyword_embeddings'
credits_df['cast_embeddings'] = credits_df['cast_names'].apply(get_avg_embedding_cast)

# Merge the embeddings with merged_df based on movieId and id
merged_df = pd.merge(merged_df, credits_df[['id', 'cast_embeddings']], left_on='movieId', right_on='id', how='left')

# Drop the redundant 'id' column after merging
merged_df.drop(columns=['id'], inplace=True)


In [8]:

# Function to extract and process director names
def extract_director(crew):
    crew_list = ast.literal_eval(crew)
    directors = [member['name'] for member in crew_list if member['job'] == 'Director']
    return directors

# Apply the function to extract director names
credits_df['director_names'] = credits_df['crew'].apply(extract_director)

# Train a Word2Vec model on director names
director_sentences = credits_df['director_names'].tolist()
director_model = Word2Vec(sentences=director_sentences, vector_size=100, window=5, min_count=1, sg=1)

# Function to get the average embedding for a list of director names
def get_avg_embedding_director(names):
    embeddings = [director_model.wv[name] for name in names]
    return np.mean(embeddings, axis=0)

# Apply the function to the director data and create a new column 'director_embeddings'
credits_df['director_embeddings'] = credits_df['director_names'].apply(get_avg_embedding_director)

# Merge the embeddings with merged_df based on movieId
merged_df = pd.merge(merged_df, credits_df[['id', 'director_embeddings']], left_on='movieId', right_on='id', how='left')


C:\Users\ssy30\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [9]:
merged_df

,userId,movieId,rating,timestamp,id_x,title,rating_mean,id_y,keyword_embeddings,cast_embeddings,id,director_embeddings
0,1,110,1.0,1425941529,110,Three Colors: Red,4.016057,110,"[-0.09102818, 0.2859355, 0.10113441, -0.021410...","[-0.053707063, 0.03131991, 0.13465041, 0.04587...",110,"[-0.0076147364, 8.7291e-05, 0.009809792, 0.006..."
1,11,110,3.5,1231676989,110,Three Colors: Red,4.016057,110,"[-0.09102818, 0.2859355, 0.10113441, -0.021410...","[-0.053707063, 0.03131991, 0.13465041, 0.04587...",110,"[-0.0076147364, 8.7291e-05, 0.009809792, 0.006..."
2,22,110,5.0,1111937009,110,Three Colors: Red,4.016057,110,"[-0.09102818, 0.2859355, 0.10113441, -0.021410...","[-0.053707063, 0.03131991, 0.13465041, 0.04587...",110,"[-0.0076147364, 8.7291e-05, 0.009809792, 0.006..."
3,24,110,5.0,979870012,110,Three Colors: Red,4.016057,110,"[-0.09102818, 0.2859355, 0.10113441, -0.021410...","[-0.053707063, 0.03131991, 0.13465041, 0.04587...",110,"[-0.0076147364, 8.7291e-05, 0.009809792, 0.006..."
4,29,110,3.0,1044020005,110,Three Colors: Red,4.016057,110,"[-0.09102818, 0.2859355, 0.10113441, -0.021410...","[-0.053707063, 0.03131991, 0.13465041, 0.04587...",110,"[-0.0076147364, 8.7291e-05, 0.009809792, 0.006..."
...,...,...,...,...,...,...,...,...,...,...,...,...
11459407,270123,151509,3.0,1453527817,151509,Trapped in the Closet: Chapters 23-33,3.000000,151509,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.05180093, 0.028097922, 0.13161632, 0.04274...",151509,"[0.003975936, 0.0032124186, 0.0024527512, -0.0..."
11459408,270123,159109,1.5,1465361347,159109,The Rambler,1.500000,159109,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.05427665, 0.023820974, 0.1516811, 0.045102...",159109,"[0.0075497795, 0.0036511684, 0.00642835, 0.009..."
11459409,270123,159109,1.5,1465361347,159109,The Rambler,1.500000,159109,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.05427665, 0.023820974, 0.1516811, 0.045102...",159109,"[0.0075497795, 0.0036511684, 0.00642835, 0.009..."
11459410,270694,165159,4.0,1476635117,165159,Agenzia Riccardo Finzi... praticamente detective,4.000000,165159,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.04848957, 0.031817306, 0.13727532, 0.04450...",165159,NaN


In [15]:

# merged_df = pd.merge(merged_df, ratings[['userId', 'movieId', 'rating']], on='movieId', how='left')
# ratings = pd.merge(ratings, merged_df[['id', 'cast_embeddings']], left_on='movieId', right_on='id', suffixes=('_ratings', '_cast'))
# ratings = pd.merge(ratings, merged_df[['id', 'director_embeddings']], left_on='movieId', right_on='id', suffixes=('_ratings', '_director'))

# ratings = pd.merge(ratings, merged_df[['id', 'keyword_embeddings', 'cast_embeddings', 'director_embeddings']], left_on='movieId', right_on='id', suffixes=('_ratings', '_keywords'))
ratings

MemoryError: Unable to allocate 1.68 TiB for an array with shape (230212913066,) and data type int64

In [17]:


X = merged_df[['userId', 'movieId', 'rating_mean', 'keyword_embeddings', 'cast_embeddings', 'director_embeddings']]

y = merged_df['rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_embeddings_flat = np.vstack(X_train[['keyword_embeddings', 'cast_embeddings', 'director_embeddings']].values)
X_test_embeddings_flat = np.vstack(X_test[['keyword_embeddings', 'cast_embeddings', 'director_embeddings']].values)

# Standardize the input features (excluding movieId and keyword_embeddings)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.drop(columns=['movieId', 'keyword_embeddings', 'cast_embeddings', 'director_embeddings' ]))
X_test_scaled = scaler.transform(X_test.drop(columns=['movieId', 'keyword_embeddings', 'cast_embeddings', 'director_embeddings']))

# Define input layers for the scaled features and the embeddings
input_scaled = Input(shape=(X_train_scaled.shape[1],), name='input_scaled')
input_embeddings = Input(shape=(X_train_embeddings_flat.shape[1],), name='input_embeddings')

# Dense layers for each input
dense_scaled = Dense(64, activation='relu')(input_scaled)
dense_embeddings = Dense(64, activation='relu')(input_embeddings)

# Concatenate the two dense layers
concatenated = Concatenate()([dense_scaled, dense_embeddings])

# Add more dense layers as needed
dense1 = Dense(32, activation='relu')(concatenated)
output = Dense(1, activation='linear')(dense1)

# Define the model with two inputs
model_nn = Model(inputs=[input_scaled, input_embeddings], outputs=output)




In [18]:

# Compile the model
model_nn.compile(optimizer='adam', loss='mean_squared_error')


In [22]:
import tensorflow as tf

# Convert embeddings arrays to tensors
X_train_embeddings_tensor = tf.convert_to_tensor(X_train_embeddings_flat.tolist())
X_test_embeddings_tensor = tf.convert_to_tensor(X_test_embeddings_flat.tolist())

# Train the model
model_nn.fit([X_train_scaled, X_train_embeddings_tensor], y_train, epochs=1, batch_size=32, verbose=0)


In [ ]:

# Make predictions
y_pred = model_nn.predict([X_test_scaled, X_test_embeddings_flat]).flatten()

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


71533/71533 [==============================] - 155s 2ms/step
Mean Squared Error: 0.9308878676359092


In [ ]:
movies

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,Animation Comedy Family,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,Adventure Fantasy Family,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,Romance Comedy,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,Comedy Drama Romance,NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,Comedy,NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,Drama Family,http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,Drama,NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,Action Drama Thriller,NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,,NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [ ]:
def get_user_recommendations(user_id, ratings, movies, model, num_recommendations=10):
    # Get movies the user has not rated
    
    rated_movies = ratings[ratings['userId'] == user_id]['movieId']
    unrated_movies = movies[~movies['id'].isin(rated_movies)]


    user_unrated = pd.DataFrame({'userId': [user_id]*len(unrated_movies), 'movieId': unrated_movies['id'].values})

    # Merge with movie metadata to get features for unrated movies
    unrated_movies_features = pd.merge(user_unrated, movies[['id', 'title']], left_on='movieId', right_on='id')

    # Merge with movie ratings mean to get mean rating for each movie
    unrated_movies_features = pd.merge(user_unrated, movies[['id', 'title']], left_on='movieId', right_on='id')
    unrated_movies_features = pd.merge(unrated_movies_features, movie_ratings, on='title')

    # Use the model to predict ratings for unrated movies
    X_pred = unrated_movies_features[['userId', 'movieId', 'rating']]
    X_pred['rating_mean'] = X_pred['rating']  # Use 'rating' as 'rating_mean' for prediction
    X_pred.drop('rating', axis=1, inplace=True)  # Drop the original 'rating' column
    unrated_movies_features['predicted_rating'] = model.predict(X_pred)

    # Sort unrated movies by predicted rating and get top recommendations
    recommendations = unrated_movies_features.sort_values(by='predicted_rating', ascending=False).head(num_recommendations)

    
    return recommendations[['title', 'predicted_rating']]


user_id = 1
recommendations = get_user_recommendations(user_id, ratings, movies, model)
print(f"Top recommendations for user {user_id}:")
print(recommendations)

# TODO: Turn keywords into embeddings, genre into embeddings, cast into embeddings, run time
# TODO: Normalize average rating on a scale of 1-max # of reviews
# Feedback: Add summary, cast, more info

C:\Users\ssy30\AppData\Local\Temp\ipykernel_38272\601170566.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_pred['rating_mean'] = X_pred['rating']  # Use 'rating' as 'rating_mean' for prediction
C:\Users\ssy30\AppData\Local\Temp\ipykernel_38272\601170566.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_pred.drop('rating', axis=1, inplace=True)  # Drop the original 'rating' column


AttributeError: 'Word2Vec' object has no attribute 'predict'